In [1]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import time

DATA_DIR = Path('../data')

test_df = pd.read_csv(DATA_DIR / 'preprocessed' / 'test.csv')
topics = pd.read_csv(DATA_DIR / 'topics.csv')

label_to_index = topics.set_index('Topic')['Number'].to_dict()

test_df['Label'] = test_df['Topic'].apply(lambda x: label_to_index[x] + 1)
test_df

,Questions,Topic,Label
0,Explain how the concept of opportunity cost ap...,entreprenuers and leaders,1
1,If nintendo reduces its console prices by 8% c...,market,3
2,Evaluate these two options and recommend which...,marketing mix and strategy,4
3,evaluate these two options and recommend whic...,meeting customer needs,5
4,Define the term ‘niche market’.,meeting customer needs,5
...,...,...,...
101,What is meant by market growth,meeting customer needs,5
102,Using this data calculate the income elasticit...,market,3
103,Analyse two non-price factors that could affec...,market,3
104,Evaluate the extent to which the non financial...,managing people,2


In [2]:
topics_for_prompt = ''
for num in topics['Number']:
    topics_for_prompt += f'{num + 1}) {topics.loc[num, "Topic"]}, '
topics_for_prompt = topics_for_prompt.rstrip(', ')  # Remove the trailing comma and space
topics_for_prompt

'1) entreprenuers and leaders, 2) managing people, 3) market, 4) marketing mix and strategy, 5) meeting customer needs'

In [3]:
import requests
import json
from time import sleep

url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Api-Key AQVNyNv0CVbumVtFMnhuEdcUURK2WHWLyIRezjJT",
}


def compose_prompt(question):
    prompt = {
        "modelUri": "gpt://b1g6odsice70le7mmg6d/yandexgpt-pro",
        "completionOptions": {
            "stream": False,
            "temperature": 0.1,
            "maxTokens": "2",
        },
        "messages": [
            {
                "role": "system",
                "text": f"""Your role is to classify British A-level exam questions into five topics: {topics_for_prompt}. Assign the appropriate topic number based on the question's theme and keywords.""",
            },
            {"role": "user", "text": "define the term brand"},
            {"role": "assistant", "text": "4"},
            {
                "role": "user",
                "text": "explain one risk jack ma may have taken when setting up the alibaba group",
            },
            {"role": "assistant", "text": "1"},
            {
                "role": "user",
                "text": "analyse two factors that may have increased demand for the alibaba group",
            },
            {"role": "assistant", "text": "3"},
            {
                "role": "user",
                "text": "discuss if profit maximisation is the main business objective of the alibaba group",
            },
            {"role": "assistant", "text": "1"},
            {
                "role": "user",
                "text": "assess the advantages of a paternalistic style of leadership for a business such as the alibaba group",
            },
            {"role": "assistant", "text": "2"},
            {"role": "user", "text": "define the term product innovation "},
            {"role": "assistant", "text": "5"},
            {
                "role": "user",
                "text": "if nintendo reduces its console prices by 8% calculate the estimated percentage change in demand. you are advised to show your working",
            },
            {"role": "assistant", "text": "3"},
            {
                "role": "user",
                "text": "analyse two different pricing strategies that nintendo might use to compete in the games console industry",
            },
            {"role": "assistant", "text": "4"},
            {
                "role": "user",
                "text": "assess the benefits of a matrix organisational structure for a business such as nintendo",
            },
            {"role": "assistant", "text": "2"},
            {
                "role": "user",
                "text": "Explain how the growth of the fitness market from 2015 to 2016 could have affected puregym",
            },
            {"role": "assistant", "text": "5"},
            {
                "role": "user",
                "text": f'Predict the topic number based on the provided question. The topics: {topics_for_prompt}. Please respond with the relevant topic NUMBER.\n{question}"',
            },
        ],
    }
    return prompt


def yagpt_inference(question, url, headers):
    sleep(1)
    prompt = compose_prompt(question)
    print(prompt)
    response = requests.post(url, headers=headers, json=prompt)
    result = response.text
    response_dict = json.loads(result)
    print(response_dict)
    text_content = response_dict["result"]["alternatives"][0]["message"]["text"]
    return text_content

In [4]:
df = test_df.sample(1)
df

,Questions,Topic,Label
85,Assess the advantages of market segmentation t...,meeting customer needs,5


In [5]:
def process_in_chunks(df, chunk_size, url, headers):
    total_chunks = (len(df) - 1) // chunk_size + 1  

    for chunk_index in range(total_chunks):
        start_index = chunk_index * chunk_size
        end_index = start_index + chunk_size
        df_chunk = df.iloc[start_index:end_index]

        tqdm.pandas(desc=f"Processing chunk {chunk_index + 1}/{total_chunks}")
        df_chunk['Predicted Label'] = df_chunk['Questions'].progress_apply(yagpt_inference, args=(url, headers))
        if chunk_index < total_chunks - 1:
            print("\nWaiting for 1 hour to avoid hitting the API limit...")
            for i in tqdm(range(3600), desc="Sleeping"):
                time.sleep(1)  # Sleep for 1 second at a time, updating the progress bar

        if chunk_index == 0:
            df_result = df_chunk
        else:
            df_result = pd.concat([df_result, df_chunk], ignore_index=True)

    return df_result

In [6]:
chunk_size = 80  # Set based on the API limit
df_result = process_in_chunks(test_df, chunk_size, url, headers)

Processing chunk 1/2:   0%|                              | 0/80 [00:00<?, ?it/s]

{'modelUri': 'gpt://b1g6odsice70le7mmg6d/yandexgpt-pro', 'completionOptions': {'stream': False, 'temperature': 0.1, 'maxTokens': '2'}, 'messages': [{'role': 'system', 'text': "Your role is to classify British A-level exam questions into five topics: 1) entreprenuers and leaders, 2) managing people, 3) market, 4) marketing mix and strategy, 5) meeting customer needs. Assign the appropriate topic number based on the question's theme and keywords."}, {'role': 'user', 'text': 'define the term brand'}, {'role': 'assistant', 'text': '4'}, {'role': 'user', 'text': 'explain one risk jack ma may have taken when setting up the alibaba group'}, {'role': 'assistant', 'text': '1'}, {'role': 'user', 'text': 'analyse two factors that may have increased demand for the alibaba group'}, {'role': 'assistant', 'text': '3'}, {'role': 'user', 'text': 'discuss if profit maximisation is the main business objective of the alibaba group'}, {'role': 'assistant', 'text': '1'}, {'role': 'user', 'text': 'assess the

Processing chunk 1/2:   2%|▌                     | 2/80 [00:01<01:15,  1.03it/s]

{'result': {'alternatives': [{'message': {'role': 'assistant', 'text': '1'}, 'status': 'ALTERNATIVE_STATUS_TRUNCATED_FINAL'}], 'usage': {'inputTextTokens': '427', 'completionTokens': '2', 'totalTokens': '429'}, 'modelVersion': '07.03.2024'}}
{'modelUri': 'gpt://b1g6odsice70le7mmg6d/yandexgpt-pro', 'completionOptions': {'stream': False, 'temperature': 0.1, 'maxTokens': '2'}, 'messages': [{'role': 'system', 'text': "Your role is to classify British A-level exam questions into five topics: 1) entreprenuers and leaders, 2) managing people, 3) market, 4) marketing mix and strategy, 5) meeting customer needs. Assign the appropriate topic number based on the question's theme and keywords."}, {'role': 'user', 'text': 'define the term brand'}, {'role': 'assistant', 'text': '4'}, {'role': 'user', 'text': 'explain one risk jack ma may have taken when setting up the alibaba group'}, {'role': 'assistant', 'text': '1'}, {'role': 'user', 'text': 'analyse two factors that may have increased demand for

Processing chunk 1/2:   4%|▊                     | 3/80 [00:10<05:23,  4.20s/it]

...

Processing chunk 2/2: 100%|█████████████████████| 26/26 [00:58<00:00,  2.01s/it]

{'result': {'alternatives': [{'message': {'role': 'assistant', 'text': '2'}, 'status': 'ALTERNATIVE_STATUS_TRUNCATED_FINAL'}], 'usage': {'inputTextTokens': '437', 'completionTokens': '2', 'totalTokens': '439'}, 'modelVersion': '07.03.2024'}}
{'modelUri': 'gpt://b1g6odsice70le7mmg6d/yandexgpt-pro', 'completionOptions': {'stream': False, 'temperature': 0.1, 'maxTokens': '2'}, 'messages': [{'role': 'system', 'text': "Your role is to classify British A-level exam questions into five topics: 1) entreprenuers and leaders, 2) managing people, 3) market, 4) marketing mix and strategy, 5) meeting customer needs. Assign the appropriate topic number based on the question's theme and keywords."}, {'role': 'user', 'text': 'define the term brand'}, {'role': 'assistant', 'text': '4'}, {'role': 'user', 'text': 'explain one risk jack ma may have taken when setting up the alibaba group'}, {'role': 'assistant', 'text': '1'}, {'role': 'user', 'text': 'analyse two factors that may have increased demand for

Processing chunk 2/2: 100%|█████████████████████| 26/26 [01:00<00:00,  2.31s/it]

{'result': {'alternatives': [{'message': {'role': 'assistant', 'text': '3'}, 'status': 'ALTERNATIVE_STATUS_TRUNCATED_FINAL'}], 'usage': {'inputTextTokens': '439', 'completionTokens': '2', 'totalTokens': '441'}, 'modelVersion': '07.03.2024'}}



/var/folders/8_/cdypz21908z67f0vvpr9mdy80000gn/T/ipykernel_38107/3780304646.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chunk['Predicted Label'] = df_chunk['Questions'].progress_apply(yagpt_inference, args=(url, headers))


In [7]:
df_result

,Questions,Topic,Label,Predicted Label
0,Explain how the concept of opportunity cost ap...,entreprenuers and leaders,1,1
1,If nintendo reduces its console prices by 8% c...,market,3,3
2,Evaluate these two options and recommend which...,marketing mix and strategy,4,2
3,evaluate these two options and recommend whic...,meeting customer needs,5,2
4,Define the term ‘niche market’.,meeting customer needs,5,3
...,...,...,...,...
101,What is meant by market growth,meeting customer needs,5,3
102,Using this data calculate the income elasticit...,market,3,3
103,Analyse two non-price factors that could affec...,market,3,3
104,Evaluate the extent to which the non financial...,managing people,2,2


In [8]:
df_result = df_result[df_result['Predicted Label'].str.isdigit()]

In [9]:
df_result['Predicted Label'] = df_result['Predicted Label'].astype(int)

In [10]:
correct_predictions = (df_result['Label'] == df_result['Predicted Label']).sum()
total_predictions = len(df_result)
accuracy = correct_predictions / total_predictions

In [11]:
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 50.94%
